## Imports

In [3]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
from dotenv import load_dotenv
from os import getenv
from test_db_con import test_mysql_conx
from pathlib import Path
load_dotenv()

False

In [14]:
CWD: Path = Path("./app/")
EXAMPLE_INPUT_PATH: Path = CWD / Path('./coding_challenge_files/example_input.txt')
TABLE_NAME = 'instruments'

## Setting up mysql database

First we test the connection to our mysql database

In [11]:
# database connection info
DB_CON_DICT = dict(
    user=getenv("MYSQL_ROOT_USER"),
    password=getenv("MYSQL_ROOT_PASSWORD"),
    host=getenv("HOST"),
    port=int(getenv("MYSQL_DOCKER_PORT")),
    database=getenv("MYSQL_DATABASE")    
)

# test database connection
test_mysql_conx(**DB_CON_DICT)

Connection Success


Then we will import the example_input.txt file as a table in our database in order to simulate a real world scenario.

In [15]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("TxtToMySQL").getOrCreate()

# Specify the path to your .txt file
txt_file_path = EXAMPLE_INPUT_PATH

# Read the .txt file into a PySpark DataFrame
df = spark.read.option("delimiter", "\t").csv(txt_file_path, header=False, inferSchema=True)

# Adjust column names as needed
df = df.toDF("INSTRUMENT_NAME","DATE","VALUE")

# Configure MySQL connection properties
mysql_properties = {
    "driver": "com.mysql.cj.jdbc.Driver",
    "url": "jdbc:mysql://{host}:{port}/{database}".format(**DB_CON_DICT), # type: ignore
    "user": DB_CON_DICT['user'],
    "password": DB_CON_DICT['password'],
}

# Write the DataFrame to MySQL
df.write.jdbc(url=mysql_properties["url"],
              table=TABLE_NAME,
              mode="overwrite",  # or "append" if needed
              properties=mysql_properties)

# Stop the Spark session
spark.stop()

JAVA_HOME is not set


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.